Save a scatter plot which compares the effect of a hyperparameter on the distribution of realized control rewards.

This script assumes that the GUILD AI library was used to generate the results.

You specify a list of hashes to compare.

Each hash references a GUILD AI experiment with e.g. 100 randomized episodes.

The randomized episodes have different initialization and seeds, so that you can be sure the effect of a hyperparameter you get is not a random outlier.

Typically, all the runs in your list used the same hyperparameters, except one that was varied.

You then have to manually define the name of the parameter and the values you gave it. (This is not extracted automatically).

The resulting plot has one row for each GUILD hash, so that e.g. 100 points in the row show the distribution of control rewards obtained during the 100 random episodes.

On the horizontal axis, you see the scale of the rewards.

In [ ]:
import os
import numpy as np
import glob
import pandas as pd

%matplotlib widget
import matplotlib.pyplot as plt
plt.style.use(["science"])

In [ ]:
if os.getcwd().split(os.sep)[-1].lower() != "controlgym": os.chdir("..")
print(f"Set working directory to {os.getcwd()}")

### List hyperparameter names and values

In [ ]:
fig_title = r"MPC Optimizer Comparison DubinsCar"
hp_name = r"optimizer"
hp_values = ["RPGD", "CEM", "CEM+Gradient Descent", "Gradient MPC", "MPPI", "Winner-Take-All"][::-1]
# # fig_title = r"RPGD Sweep Population Size CartPoleSimulator"
# # hp_name = "number of parallel rollouts (total / kept when resampling)"
# # hp_values = ["1/1","2/1","4/1","8/2","16/4","32/0","32/8","32/32","64/16","128/32","256/64",]
# fig_title = r"RPGD Sweep Resampling Interval CartPoleSimulator"
# hp_name = "number of control iterations between resampling of input plans"
# hp_values = [1,5,10,20,50,200]
# fig_title = r"RPGD Sweep Gradient Steps Dubins Car"
# hp_name = "gradient updates per control step"
# hp_values = [0,1,2,5,10,25,50,75,100,150]

### List GUILD AI hashes of runs to plot
GUILD_AI_HASHES = [
    "ea42e4a0",
    "30dc41fb",
    "93e932f9",
    "904ff4d8",
    "d3a3f510",
    "f2cae2a6",
]

### Make sure that ".env" below is the path to your python virtual environment
PATHS_TO_EXPERIMENTS = [os.path.join(".env", ".guild", "runs", h) for h in GUILD_AI_HASHES]

### Or specify paths manually, like so:
# PATHS_TO_EXPERIMENTS = [
#     ".env/.guild/runs/4c8c1726",
# ]

### Collect Rewards as Dataframe

In [ ]:
num_experiments = len(PATHS_TO_EXPERIMENTS)
all_rewards_data = []
all_run_result_data = {"timeout": [], "terminated": [], "truncated": []}

for path_to_experiment in PATHS_TO_EXPERIMENTS:
    output_scalars_files = glob.glob(f"{path_to_experiment}*{os.sep}Output{os.sep}**{os.sep}*output_scalars*.csv", recursive=True)
    assert len(output_scalars_files) == 1
    
    output_scalars_of_experiment = np.loadtxt(
        output_scalars_files[0],
        dtype=np.float32,
        delimiter=",",
        skiprows=1,
    )
    
    rewards_of_experiment = output_scalars_of_experiment[:, 0]
    all_run_result_data["timeout"].append(output_scalars_of_experiment[:, 1])
    all_run_result_data["terminated"].append(output_scalars_of_experiment[:, 2])
    all_run_result_data["truncated"].append(output_scalars_of_experiment[:, 3])
    all_rewards_data.append(rewards_of_experiment)
    
all_rewards_data = np.stack(all_rewards_data, axis=0)  # One row for each experiment
all_run_result_data = {k: np.stack(v, axis=1) for k, v in all_run_result_data.items()}
num_trials = all_rewards_data.shape[1]

In [ ]:
{k: np.mean(v, axis=0) for k, v in all_run_result_data.items()}

In [ ]:
[np.mean(a) for a in all_rewards_data]

### Plot Outputs

#### Horizontal Layout

In [ ]:
plt.close("all")
fig, ax = plt.subplots(
    figsize=(5.5, 2.5),
    dpi=300.0,
)

for i in range(num_experiments):
    scatter_x = np.repeat(i + 1, num_trials) + np.clip(0.07 * np.random.standard_normal(num_trials), -0.35, 0.35)
    r = all_rewards_data[i, :]

    ax.scatter(scatter_x, r, marker=".", c="b", s=4)
    ax.plot([i + 1 - 0.35, i + 1 + 0.35], [np.mean(r), np.mean(r)], c="r", linewidth=0.75)

ax.grid(visible=True, which="major", axis="y", lw=0.3)
ax.set_xlabel(hp_name, fontsize=12)
ax.set_ylabel("mean reward per episode", fontsize=12)
ax.set_xticks(np.arange(1, num_experiments + 1), labels=hp_values)
ax.minorticks_off()
# ax.set_title(fig_title, fontdict={"fontsize": 9}, pad=3.0)
# ax.spines[['top', 'right']].set_visible(False)
fig.tight_layout(pad=1.03)

#### Vertical Layout

In [ ]:
plt.close("all")
fig, ax = plt.subplots(
    figsize=(4, 3),
    dpi=300.0,
)

all_x = []
all_y = []

for i in range(num_experiments):
    scatter_y = np.repeat(i + 1, num_trials) + np.clip(0.07 * np.random.standard_normal(num_trials), -0.35, 0.35)
    r = all_rewards_data[i, :]
    
    all_x.extend(list(r))
    all_y.extend(list(scatter_y))

    colors = ["r" if to or tr else "g" for to, tr in zip(all_run_result_data["timeout"][:, i], all_run_result_data["truncated"][:, i])]
    ax.scatter(r, scatter_y, marker=".", c=colors, s=4)
    ax.plot([np.mean(r), np.mean(r)], [i + 1 - 0.35, i + 1 + 0.35], c="b", linewidth=0.75)

ax.grid(visible=True, which="major", axis="x", lw=0.3)
# ax.set_xlabel("mean reward per episode", fontsize=8)
# ax.set_ylabel(hp_name)
# ax.set_yticks(np.arange(1, num_experiments + 1), labels=hp_values)
ax.set_xticklabels([])
ax.set_yticklabels([])
# ax.set_title(fig_title, fontdict={"fontsize": 9}, pad=3.0)
# ax.spines[['top', 'right']].set_visible(False)
ax.minorticks_off()
fig.tight_layout(pad=1.03)

## Saving

In [ ]:
fig.savefig(os.path.join(fig_title + "_" + "_".join(GUILD_AI_HASHES) + ".pdf"), bbox_inches="tight", pad_inches=0.03)

In [ ]:
savedir = os.path.join("Output", "cost_scatter_plots")
if not os.path.exists(savedir):
    os.makedirs(savedir)

### Save Figure

In [ ]:
fig.savefig(os.path.join(savedir, fig_title + "_" + "_".join(GUILD_AI_HASHES) + ".pdf"), bbox_inches="tight", pad_inches=0.03)

### Save data as .dat file

In [ ]:
df = pd.DataFrame({
    "xcol": all_x,
    "ycol": all_y,
    "metacol": all_x,
})
df.to_csv(os.path.join(savedir, fig_title + "_" + "_".join(GUILD_AI_HASHES) + ".dat"), sep="\t", index=False)